In [1]:
# import required libraries
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time


  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


In [2]:
# download the shakespeare dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 1s 1us/step


In [3]:
# read the data
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# take a look at first 250 chars
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# the unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [7]:
# process the text with vectors

# create a mapping from unqiue characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
# now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to len(unique)
print('{')
for char, _ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...



In [10]:
# show how the first 13 characters from the text are mappers to integers
print('{} ----- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ----- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [12]:
# the prediction task

# given a sequence of characters, what is the most probably next character? 

# This is the task we are training our model to perform


# the max length sequence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [13]:
# batch method lets us easily convert these individual characters to sequences of the desired size
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [15]:
# for each sequence duplicate and shift it to form the input and target text by using the map 
# method to apply a simple function to each batch

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [17]:
# print the first examples input and target values
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [19]:
# each index of these vectors are processed as one time step. 
# For the input at time step 0, the model receives the index for "F" and tries to predict the index for "i" as the next character.
# At the next timestep, it does the same thing but RNN considers the previous step context in addition to the current input character

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))      
    

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [20]:
# create training batches

# we use tf.data to split the etxt into manageable sequences. 
# But before feeding this data into the model, we need to shuffle the data and pack it into batches

# batch size
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possible infinite sequences
# so it doesn't attempt to shuffle the sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements)

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [21]:
# build the model

# length of the vocab in chars
vocab_size = len(vocab)

# the embedding dimension
embedding_dim = 256

# number of RNN units
rnn_units = 1024

In [22]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation='sigmoid'
    )

In [24]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(
            vocab_size,
            embedding_dim,
            batch_input_shape=[batch_size, None]
        ),
        rnn(
            rnn_units,
            return_sequences=True,
            recurrent_initializer='glorot_uniform',
            stateful=True
        ),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [25]:
model = build_model(
    vocab_size= len(vocab),
    embedding_dim= embedding_dim,
    rnn_units= rnn_units,
    batch_size=BATCH_SIZE
)

In [26]:
# run the model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [29]:
# now to try for the first example in the batch
sampled_indices = tf.random.categorical(
    example_batch_predictions[0],
    num_samples=1
)

sampled_indices = tf.squeeze(
    sampled_indices,
    axis=-1
).numpy

sampled_indices

<bound method _EagerTensorBase.numpy of <tf.Tensor: id=7607, shape=(100,), dtype=int64, numpy=
array([ 2, 13, 16, 20, 39, 40,  7, 26, 59, 58,  8,  9,  4, 33, 51, 44, 44,
       21, 52,  3,  1, 35, 57, 25, 27, 27, 31, 36, 42, 42, 51, 53, 62, 25,
       57, 11,  6,  3, 49, 36, 46, 49, 21, 17,  9, 22, 22, 53, 62, 47,  0,
       64, 21, 16, 30, 61, 45, 45,  4, 15, 50, 10,  1, 31, 26, 19, 63, 42,
       57, 35, 52, 50, 26, 26, 25,  9,  3, 57,  8, 12,  5, 35, 53,  9, 61,
       44, 55, 19, 22, 16, 21, 43, 60, 52,  8, 37, 29, 26, 30, 54],
      dtype=int64)>>

In [45]:
# decode the above to see the text predicted by this untrained model
print('Input: \n', repr("".join(idx2char[input_example_batch[0]])))
print()
print('Next Char Predictions: \n', repr("".join(idx2char[sampled_indices()])))

Input: 
 'th as this? No, you are deceived;\ntherefore, back to Rome, and prepare for your\nexecution: you are c'

Next Char Predictions: 
 "!ADHab-Nut.3&UmffIn$ WsMOOSXddmoxMs;,$kXhkIE3JJoxi\nzIDRwgg&Cl: SNGydsWnlNNM3$s.?'Wo3wfqGJDIevn.YQNRp"


In [47]:
# train the model

# attach an optimiser and loss function

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar loss:      ", example_batch_loss.numpy().mean())


Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar loss:       4.1747603


In [48]:
model.compile(
    optimizer= tf.train.AdamOptimizer(),
    loss = loss
)

In [49]:
# configure checkpoints

# directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only=True
)

In [50]:
# execute the training

# to keep training time reasonable epochs is set to 3
EPOCHS = 3

history = model.fit(
    dataset.repeat(),
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    callbacks=[checkpoint_callback]
)

Epoch 1/3
174/174 [==============================] - 1090s 6s/step - loss: 2.7118
Epoch 2/3
174/174 [==============================] - 1100s 6s/step - loss: 1.9411
Epoch 3/3
174/174 [==============================] - 1180s 7s/step - loss: 1.6817


In [51]:
# generate text

# restore the latest checkpoint

# to keep this prediction step simple, use a batch size of 1
# because of how RNN stateis passed from timestep, the model only accepts a fixed batch size once built
# to run the model with a different batch_size, we need to rebuild the model and restore the weights from the
# checkpoint

tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_3'

In [52]:
# load the weights from the checkpoint
model = build_model(
    vocab_size,
    embedding_dim,
    rnn_units,
    batch_size=1
)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1,None]))

In [53]:
# show the summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [56]:
# the prediction loop

# starts by choosing a start string, initializing the RNN state and setting the number of characters to generate

# get the prediction distribution of the next character using the start string and the RNN state

# use a multinomial distribution to calculate the index of the predicted character. Use this predicted character
# as our next input to the model

# the RNN state returned by the model is fed back into the model so that it now has more context, instead of only one word.
# After predicting the next word, the modified RNN states are again fed back into the model. Which is how it learns as it 
# gets more context from the previously predicted words

def generate_text(model, start_string):
    
    # evaluation step (generating text using learned model)
    
    # Number of characters to generate
    num_generate = 1000
    
    # Converting our starting string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    # Empty string to store our results
    text_generated = []
    
    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting
    temperature = 1.
    
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
        
        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [58]:
print(generate_text(model, start_string=u"Tom thou"))

Tom thou shall parnch
I will I suppoth, best wede-in assears.

BUCKINGHAM:

GLOUCESTER:

CORIOLANUS:
to shall oppecond thee begantw's distake, your feath. O, what's
he is not is yourgell me and exerpse,
Then ke propship their ted you daughter.
Fwry provise.

JIHANTE:
I the desirmss of oun if, I spay him huph,
And for if that sid, why dower will be sunting cewelthip,
And bound know live! I'll nat up a condrieve
E culse thy faber word, what I wnamble will live.

BINCOFFOLNCI:
With our peace fuch were perife:
Then that I shall bod
Thy frie distignines if in ness now,
I dase you gun to beagle.

IO:
And filly bray in the noth with orrelss, the not
Mate no love of our swrine; by nor shoming thy
Orfuring; a douch'd
And fellows siir; do note, betome, dequesing his Tapliear,
The sun to beeath came your bey provess; I heart my feed,
Thought upsen the turtie, the oftrown, thou, Great My with wom, downing you; from the gardan meant,
Whather he world gove here for his wed.
You what I am our prisces

In [60]:
## Customised Training

# the above procedure is simple but does not give you much control

# we have seen how to runt he model manually, let's unpack the training loop and implement it ourselves.
# This gives a starting point, for example, to implement the curriculum learning to help stabilize the model's open-loop output.

# We will use tf.GradientTape to track the gradients. You can learn more about this approach by reading the eager execution guide.

# The procedure works as follows:
#
# First, initalize the RNN state. We do this by calling the tf.keras.Model.reset_states
#
# Next, iterate over the dataset (batch by batch) and calculate the predictions associated with each.
#
# Open a tf.GradientTape, and calculate the predictions and loss in that context.
#
# Calculate the gradients of the loss with respect to the model variables using the tf.GradientTape.grads method
#
# Finally, take a step downwards by using the optimizer's tf.train.Optimizer.apply_gradients method

model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size=BATCH_SIZE
)

In [61]:
# optimiser

optimizer = tf.train.AdamOptimizer()

In [63]:
# training step
EPOCHS = 1

for epoch in range(EPOCHS):
    
    start = time.time()
    
    # initializing the hidden state at the start of every epoch
    # initially hidden is None
    hidden = model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            
            # feeding the hidden state back into the model
            # This is the interesting step
            predictions = model(inp)
            loss = tf.losses.sparse_softmax_cross_entropy(target, predictions)
            
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {:.4f}'
            print(template.format(epoch+1, batch_n, loss))
            
    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
        
    print('Epoch {} Loss {:.4f}'.format(time.time() - start))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.1739


KeyboardInterrupt: 